In [1]:
%pwd

'C:\\Users\\YUI2KOR\\BOSCH\\KNOWLEDGE\\NOTEBOOK\\notebooks\\deplearning.ai\\event_driven_agentic_document_workflows'

In [6]:
from utils import get_google_api_key

In [7]:
google_api_key = get_google_api_key()

In [11]:
import google.generativeai as genai

# Set your Gemini API Key
genai.configure(api_key=google_api_key)

# Initialize embedding model
result = genai.embed_content(
    model="models/embedding-001",
    content="What is the meaning of life?"
)

print(result['embedding'])  # Extract embedding vector

RetryError: Timeout of 60.0s exceeded, last exception: 503 failed to connect to all addresses; last error: UNKNOWN: ipv4:127.0.0.1:3128: tcp handshaker shutdown

In [1]:
import json
import os
from llama_index.core import VectorStoreIndex, SimpleDirectoryReader, ServiceContext
from llama_index.embeddings.google import GeminiEmbedding
from llama_index.llms.google import Gemini
from llama_index.node_parser import SimpleNodeParser
from qdrant_client import QdrantClient

ModuleNotFoundError: No module named 'llama_index.embeddings.google'

In [3]:
import json
import os
from llama_index.core import VectorStoreIndex, SimpleDirectoryReader, ServiceContext
# from llama_index.embeddings.langchain import LangchainEmbedding
from langchain.embeddings import GoogleGenerativeAIEmbeddings
from llama_index.llms.langchain import LangChainLLM
from langchain.chat_models import ChatGoogleGenerativeAI
from qdrant_client import QdrantClient

ImportError: cannot import name 'GoogleGenerativeAIEmbeddings' from 'langchain.embeddings' (C:\Users\YUI2KOR\BOSCH\KNOWLEDGE\NOTEBOOK\env_notebook\lib\site-packages\langchain\embeddings\__init__.py)

In [ ]:


# Set up Gemini API Key
os.environ["GOOGLE_API_KEY"] = "your-gemini-api-key"

# Read JD from a text file
def read_jd_file(file_path):
    with open(file_path, "r", encoding="utf-8") as file:
        return file.read()

# Extract key skills using LlamaIndex + Gemini
def extract_key_skills(jd_text):
    # Initialize LlamaIndex components with Gemini
    embed_model = GeminiEmbedding()
    llm = Gemini(model="gemini-pro")  # Use Gemini-Pro for LLM processing
    service_context = ServiceContext.from_defaults(llm=llm, embed_model=embed_model)

    # Convert JD text into indexed format
    parser = SimpleNodeParser()
    nodes = parser.get_nodes_from_documents([jd_text])

    # Create an index
    index = GPTVectorStoreIndex(nodes, service_context=service_context)

    # Query the index for key skills
    query = "Extract the key skills required for this job description and format them in JSON."
    response = index.query(query)
    
    return response.response  # Extracted JSON

# Store extracted skills in Qdrant (optional)
def store_in_qdrant(data):
    client = QdrantClient("localhost", port=6333)
    client.upload_collection(
        collection_name="job_skills",
        documents=[{"id": 1, "payload": data}]
    )

# Main function
if __name__ == "__main__":
    file_path = "job_description.txt"  # Path to JD text file
    jd_text = read_jd_file(file_path)

    key_skills_json = extract_key_skills(jd_text)

    # Convert response to JSON format
    skills_data = json.loads(key_skills_json)

    # Print formatted JSON
    print(json.dumps(skills_data, indent=4))

    # Store in Qdrant (optional)
    store_in_qdrant(skills_data)
